In [1]:
import os

#print(os.listdir("../../data/raw"))
#assert False
from bs4 import BeautifulSoup  
import requests
import zipfile
import io

import pandas as pd
import numpy as np
import tqdm

In [2]:
def get_download_href(soup):
    temp_finder = soup
    download_href = temp_finder.find_all("a", text=" fonts.zip")[0]["href"]
    return download_href

def get_download_href_by_url(url):
    # just for uci's situation
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    download_href = get_download_href(soup)
    return url + "/" + download_href

def create_zip_data_folder(path_folder, folder_name_zip_data):
    path_folder_zip_data = os.path.join(path_folder, folder_name_zip_data)
    os.makedirs(path_folder_zip_data)
    return path_folder_zip_data

def extract_download_href_to_zip_data_folder(download_href, path_zipped_folder_zipcode_shapefile):
    r = requests.get(download_href)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(path_zipped_folder_zipcode_shapefile)

def delete_zip_data_folder(path_folder_zip_data):
    for file_name in os.listdir(path_folder_zip_data):
        # construct full file path
        file = os.path.join(path_folder_zip_data, file_name)
        if os.path.isfile(file):
            os.remove(file)
    os.rmdir(path_folder_zip_data)

def create_fonts_csv(path_folder_zip_data, path_file_fonts):
    list_font_file_names = os.listdir(path_folder_zip_data)

    num_font_csvs = len(list_font_file_names)

    df_init = pd.read_csv(os.path.join(path_folder_zip_data, list_font_file_names[0]))

    pd_concat_df_list = [df_init]
    corrupt_fonts = []

    for i in tqdm.tqdm(range(1, num_font_csvs)):
        temp_file_name_font = list_font_file_names[i]
        temp_path_folder_font = os.path.join(path_folder_zip_data, temp_file_name_font)
        temp_font_df_orig = pd.read_csv(temp_path_folder_font)
        try:
            assert (df_init.shape[1] == temp_font_df_orig.shape[1])
            assert (list(df_init.columns) == list(temp_font_df_orig.columns))
            pd_concat_df_list.append(temp_font_df_orig)
        except:
            print(temp_file_name_font)
            corrupt_font = temp_file_name_font
            corrupt_fonts.append(corrupt_font)
            pass

    pd_concat_df = pd.concat(pd_concat_df_list, axis=0)
    fonts_df = pd_concat_df
    fonts_df.to_csv(path_file_fonts, index=False)

In [ ]:
import json 
      
# Data to be written 
dictionary ={ 
  "id": "04", 
  "name": "sunil", 
  "department": "HR"
} 
      
# Serializing json  
json_object = json.dumps(dictionary, indent = 4) 
print(json_object)

In [3]:

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00417/"
folder_name_zip_data = "fonts"
path_folder = "../../data/raw"
file_name_fonts = "fonts.csv"

download_href = get_download_href_by_url(url)
path_folder_zip_data = create_zip_data_folder(path_folder, folder_name_zip_data)
extract_download_href_to_zip_data_folder(download_href, path_folder_zip_data)
path_file_fonts = os.path.join(path_folder, file_name_fonts)
create_fonts_csv(path_folder_zip_data, path_file_fonts)
delete_zip_data_folder(path_folder_zip_data)


100%|████████████████████████████████████████████████████████████████████████████████| 152/152 [01:39<00:00,  1.52it/s]
